In [1]:
import pandas as pd
import numpy as np

In [24]:
data = pd.read_csv("../data/processed/period/ecmen/6_16/ecmenperiod1.csv.gz",compression='gzip',parse_dates=["Trade_Datetime","ecmen_public_time"],index_col=["Trade_Datetime"])

In [25]:
weather_data = pd.read_csv("../data/processed/WeatherData/ecmen_weather_subclass.csv", parse_dates=["Trans_INIT_Time"])

In [26]:
data.head(5)

,Contract_Delivery_Date,Vwap,ecmen_public_time,dst_flag,Normal_Vwap
Trade_Datetime,,,,,
2015-01-28 13:29:00,1503,2.843,2015-01-28 01:40:00,0,1.000
2015-01-28 13:30:00,1503,2.843,2015-01-28 01:40:00,0,1.000
2015-01-28 13:31:00,1503,2.839,2015-01-28 01:40:00,0,0.999
2015-01-28 13:32:00,1503,2.838,2015-01-28 01:40:00,0,0.998
2015-01-28 13:33:00,1503,2.837,2015-01-28 01:40:00,0,0.998


In [27]:
weather_data.head()

,Trans_INIT_Time,VALUE_hdd,10Y_NORMAL_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,10Y_NORMAL_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd
0,2015-01-01 01:40:00,34.32,28.51,451.82,0.00,0.00,0.19,0.19,2.70,0.00,0.00
1,2015-01-01 13:40:00,31.67,28.67,444.46,-2.95,-7.40,0.33,0.17,2.55,-0.09,0.02
2,2015-01-02 01:40:00,30.88,28.67,453.45,8.99,10.39,0.31,0.17,2.67,0.12,0.04
3,2015-01-02 13:40:00,27.33,28.70,448.02,-4.52,-4.38,0.49,0.16,2.59,0.10,0.11
4,2015-01-03 01:40:00,27.17,28.70,462.12,14.10,7.93,0.47,0.16,2.42,-0.17,-0.19


In [28]:
res = data.groupby(['Contract_Delivery_Date', pd.Grouper(freq='D')]).agg(ecmen_public_time=('ecmen_public_time', 'first'),dst_flag=('dst_flag', 'first'), Normal_vwap=('Normal_Vwap', 'last'))

In [9]:
res

ecmen_public_time  dst_flag  \
Contract_Delivery_Date Trade_Datetime                                 
1503                   2015-01-28     2015-01-28 01:40:00         0   
                       2015-01-29     2015-01-29 01:40:00         0   
                       2015-01-30     2015-01-30 01:40:00         0   
                       2015-02-02     2015-02-02 01:40:00         0   
                       2015-02-03     2015-02-03 01:40:00         0   
...                                                   ...       ...   
2104                   2021-03-26     2021-03-26 02:40:00         1   
                       2021-03-29     2021-03-29 02:40:00         1   
2105                   2021-03-29     2021-03-29 02:40:00         1   
                       2021-03-30     2021-03-30 02:40:00         1   
                       2021-03-31     2021-03-31 02:40:00         1   

                                       Normal_vwap  
Contract_Delivery_Date Trade_Datetime               
1503                   2015-01-28            0.999  
                       2015-01-29            0.953  
                       2015-01-30            1.011  
                       2015-02-02            1.005  
                       2015-02-03            1.018  
...                                            ...  
2104                   2021-03-26            1.002  
                       2021-03-29            1.020  
2105                   2021-03-29            0.997  
                       2021-03-30            0.979  
                       2021-03-31            0.991  

[1620 rows x 3 columns]

In [54]:
merge_data = pd.merge(res, weather_data, how="inner",left_on='ecmen_public_time', right_on='Trans_INIT_Time',
                      suffixes=('_hdd', '_cdd'))
merge_data

,ecmen_public_time,dst_flag,Normal_vwap,Trans_INIT_Time,VALUE_hdd,10Y_NORMAL_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,10Y_NORMAL_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd
0,2015-01-28 01:40:00,0,0.999,2015-01-28 01:40:00,29.86,29.87,457.44,-1.94,0.60,0.01,0.22,0.89,-0.04,0.00
1,2015-01-29 01:40:00,0,0.953,2015-01-29 01:40:00,27.87,29.74,462.88,-0.06,2.03,0.01,0.22,0.77,-0.05,-0.13
2,2015-01-30 01:40:00,0,1.011,2015-01-30 01:40:00,31.69,29.67,454.05,-15.93,-5.05,0.04,0.23,0.75,-0.19,-0.16
3,2015-02-02 01:40:00,0,1.005,2015-02-02 01:40:00,35.62,29.48,442.86,-12.57,-8.94,0.12,0.26,0.57,0.13,0.03
4,2015-02-03 01:40:00,0,1.018,2015-02-03 01:40:00,35.89,29.32,436.36,-14.11,-7.14,0.03,0.27,0.42,0.08,-0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,2021-03-26 02:40:00,1,1.002,2021-03-26 02:40:00,12.97,16.58,202.84,7.30,1.79,1.12,0.88,9.34,-0.94,-0.27
1616,2021-03-29 02:40:00,1,1.020,2021-03-29 02:40:00,16.00,15.82,188.04,-6.12,-2.65,0.53,0.97,9.49,-0.02,0.15
1617,2021-03-29 02:40:00,1,0.997,2021-03-29 02:40:00,16.00,15.82,188.04,-6.12,-2.65,0.53,0.97,9.49,-0.02,0.15
1618,2021-03-30 02:40:00,1,0.979,2021-03-30 02:40:00,12.96,15.52,182.09,-1.19,-0.23,1.05,0.98,10.79,-0.39,-0.37


In [55]:
merge_data["Month"] = merge_data["Trans_INIT_Time"].dt.month

In [56]:
merge_data.drop(columns=['Trans_INIT_Time','dst_flag','10Y_NORMAL_hdd','10Y_NORMAL_cdd'],inplace=True)

In [57]:
merge_data

,ecmen_public_time,Normal_vwap,VALUE_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd,Month
0,2015-01-28 01:40:00,0.999,29.86,457.44,-1.94,0.60,0.01,0.89,-0.04,0.00,1
1,2015-01-29 01:40:00,0.953,27.87,462.88,-0.06,2.03,0.01,0.77,-0.05,-0.13,1
2,2015-01-30 01:40:00,1.011,31.69,454.05,-15.93,-5.05,0.04,0.75,-0.19,-0.16,1
3,2015-02-02 01:40:00,1.005,35.62,442.86,-12.57,-8.94,0.12,0.57,0.13,0.03,2
4,2015-02-03 01:40:00,1.018,35.89,436.36,-14.11,-7.14,0.03,0.42,0.08,-0.02,2
...,...,...,...,...,...,...,...,...,...,...,...
1615,2021-03-26 02:40:00,1.002,12.97,202.84,7.30,1.79,1.12,9.34,-0.94,-0.27,3
1616,2021-03-29 02:40:00,1.020,16.00,188.04,-6.12,-2.65,0.53,9.49,-0.02,0.15,3
1617,2021-03-29 02:40:00,0.997,16.00,188.04,-6.12,-2.65,0.53,9.49,-0.02,0.15,3
1618,2021-03-30 02:40:00,0.979,12.96,182.09,-1.19,-0.23,1.05,10.79,-0.39,-0.37,3


In [58]:
gn_data = pd.read_csv("../data/processed/NgSdData/gn sd.csv",parse_dates=["date"])

In [59]:
gn_data['DATE_DAY'] = gn_data['date'].dt.date

In [60]:
merge_data['DATE_DAY'] = merge_data['ecmen_public_time'].dt.date

In [61]:
merge_data = merge_data.merge(gn_data, on='DATE_DAY')
merge_data.drop(columns=['DATE_DAY'],inplace=True)

In [62]:
merge_data

,ecmen_public_time,Normal_vwap,VALUE_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd,...,vs5y Power Burn,vs5y Net Burn,vs5y Net Import,vs5y Storage Level,Production S1,Production S2,Net Import,Power Burn,Net Burn,Adjusted Balance
0,2015-02-27 01:40:00,1.010,42.30,421.13,-4.10,-1.11,0.08,3.48,-0.20,-0.13,...,-0.1,0.7,-0.6,-143.2,-0.2,-0.4,0.4,0.2,-0.7,8.1
1,2015-03-02 01:40:00,0.999,33.08,373.25,5.25,2.37,0.30,4.03,0.12,0.16,...,1.7,1.9,-1.9,-195.6,0.4,0.4,-1.0,2.5,2.2,8.1
2,2015-03-03 01:40:00,1.021,29.48,347.40,-12.13,-6.29,0.34,4.64,0.42,0.37,...,3.3,4.3,-2.3,-200.5,0.5,0.4,-1.3,3.5,4.1,8.1
3,2015-03-04 01:40:00,1.011,29.99,332.14,6.96,3.24,0.50,4.99,-0.16,-0.16,...,6.6,7.8,-2.4,-198.1,0.1,-0.4,-0.9,6.1,6.7,8.1
4,2015-03-05 01:40:00,1.024,38.23,321.77,1.70,5.83,0.41,4.72,0.32,0.04,...,6.1,6.2,-1.0,-202.9,0.0,-1.1,0.5,4.6,4.0,8.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,2021-03-26 02:40:00,1.002,12.97,202.84,7.30,1.79,1.12,9.34,-0.94,-0.27,...,1.5,1.5,-10.6,-34.9,-0.5,-0.2,-0.5,1.7,1.4,4.6
1594,2021-03-29 02:40:00,1.020,16.00,188.04,-6.12,-2.65,0.53,9.49,-0.02,0.15,...,-1.7,1.2,-10.2,-22.6,0.6,0.5,0.1,-2.1,0.9,4.6
1595,2021-03-29 02:40:00,0.997,16.00,188.04,-6.12,-2.65,0.53,9.49,-0.02,0.15,...,-1.7,1.2,-10.2,-22.6,0.6,0.5,0.1,-2.1,0.9,4.6
1596,2021-03-30 02:40:00,0.979,12.96,182.09,-1.19,-0.23,1.05,10.79,-0.39,-0.37,...,-1.6,0.4,-11.0,-22.5,0.7,0.6,-0.5,-1.9,0.0,4.6


In [63]:
file_name = "ecmen_period1_ngsd_no_dst"+".csv"
merge_data.to_csv(file_name,index=False, float_format='%.3f')